In [1]:
import sys
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
file = Path("../stimage").resolve()
parent= file.parent
sys.path.append(str(parent))
from PIL import Image
from stimage._utils import gene_plot, Read10X, ReadOldST, tiling, ensembl_to_id
from stimage._model import CNN_NB_multiple_genes, negative_binomial_layer, negative_binomial_loss
from stimage._data_generator import DataGenerator
import tensorflow as tf
import seaborn as sns
sns.set_style("white")
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
# import geopandas as gpd
from sklearn.neighbors import KDTree
from anndata import read_h5ad
from tensorflow.keras import backend as K
import scanpy as sc

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [3]:
import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from splot.esda import moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation

In [4]:
from scipy import stats

def plot_correlation(df, attr_1, attr_2):
    r = stats.pearsonr(df[attr_1], 
                       df[attr_2])[0] **2

    g = sns.lmplot(data=df,
        x=attr_1, y=attr_2,
        height=5, legend=True
    )
    # g.set(ylim=(0, 360), xlim=(0,360))

    g.set_axis_labels(attr_1, attr_2)
    plt.annotate(r'$R^2:{0:.2f}$'.format(r),
                (max(df[attr_1])*0.9, max(df[attr_2])*0.9))
    return g


def calculate_correlation(attr_1, attr_2):
    r = stats.pearsonr(attr_1, 
                       attr_2)[0]
    return r

def calculate_correlation_2(attr_1, attr_2):
    r = stats.spearmanr(attr_1, 
                       attr_2)[0]
    return r

In [5]:
DATA_PATH = Path("/clusterdata/uqxtan9/Q1851/Xiao/Working_project/her2st")

In [6]:
adata_all = read_h5ad(DATA_PATH / "all_adata.h5ad")

# adata_all = ensembl_to_id(adata_all)

samples = adata_all.obs["library_id"].unique().tolist()

gene_list=["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1",
           "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

In [7]:
adata_all.obs["X"] = [x for x,y in adata_all.obs.index.str.split("x")]
adata_all.obs["Y"] = [y.split("-")[0] for x,y in adata_all.obs.index.str.split("x")]

In [8]:
adata_all.obs.library_id == adata_all.obs.library_id.unique()[0]

10x22-C2     True
10x23-C2     True
11x21-C2     True
11x22-C2     True
11x23-C2     True
            ...  
9x23-D3     False
9x24-D3     False
9x25-D3     False
9x26-D3     False
9x27-D3     False
Name: library_id, Length: 12584, dtype: bool

In [9]:
adata_all.obs

,imagecol,imagerow,tile_path,tile_tissue_mask_path,tissue_area,library_id,X,Y
10x22-C2,2537.89,6112.46,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/C2_tissue_mask/C2-2537.89-6112.46-299.jpeg,0.985537,C2,10,22
10x23-C2,2535.86,6396.68,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/C2_tissue_mask/C2-2535.86-6396.68-299.jpeg,0.891332,C2,10,23
11x21-C2,2839.29,5824.46,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/C2_tissue_mask/C2-2839.29-5824.46-299.jpeg,0.994899,C2,11,21
11x22-C2,2854.07,6104.30,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/C2_tissue_mask/C2-2854.07-6104.3-299.jpeg,0.990157,C2,11,22
11x23-C2,2847.41,6388.81,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/C2_tissue_mask/C2-2847.41-6388.81-299.jpeg,1.000000,C2,11,23
...,...,...,...,...,...,...,...,...
9x23-D3,2343.72,6411.11,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/D3_tissue_mask/D3-2343.72-6411.11-299.jpeg,1.000000,D3,9,23
9x24-D3,2338.20,6707.19,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/D3_tissue_mask/D3-2338.2-6707.19-299.jpeg,1.000000,D3,9,24
9x25-D3,2345.75,6992.77,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/D3_tissue_mask/D3-2345.75-6992.77-299.jpeg,1.000000,D3,9,25
9x26-D3,2332.38,7289.73,/clusterdata/uqxtan9/Q1851/Xiao/Working_projec...,/tmp/D3_tissue_mask/D3-2332.38-7289.73-299.jpeg,1.000000,D3,9,26


In [10]:
max_x = adata_all.obs.X.astype(int).max()
max_y = adata_all.obs.Y.astype(int).max()

In [11]:
max_x

32

In [12]:
max_y

34

In [13]:
# import numpy as np
# from tensorflow import keras
# from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
# from tensorflow.keras.preprocessing import image
# from stimage._imgaug import seq_aug
# from sklearn.neighbors import KDTree


# class DataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'

#     def __init__(self, adata, dim=(299, 299), n_channels=3, genes=None, aug=False, tile_path="tile_path"):
#         'Initialization'
#         self.dim = dim
#         self.adata = adata
#         self.n_channels = n_channels
#         self.genes = genes
#         self.num_genes = len(genes)
#         self.aug = aug
#         self.tile_path = tile_path
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return len(self.adata.obs.library_id.unique())

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Find list of IDs
#         obs_temp = self.adata.obs_names[self.adata.obs.library_id == self.adata.obs.library_id.unique()[index]]

#         # Generate data
#         X_img = self._load_img(obs_temp)
#         y = self._load_label(obs_temp)
#         pos_x,pos_y = self._load_position(obs_temp)
        
#         return {"input_img":X_img, "x":pos_x, "y":pos_y}, y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.adata.obs.library_id.unique()))

#     def _load_img(self, obs):
#         img_paths = self.adata.obs.loc[obs, 'tile_path']
#         X_img = np.zeros(shape=(len(img_paths),299,299,3))
#         for i, img_path in enumerate(img_paths):
#             X_img_ = image.load_img(img_path, target_size=self.dim)
#             X_img[i] = image.img_to_array(X_img_).astype('uint8')
#             #         X_img = np.expand_dims(X_img, axis=0)
#             #         n_rotate = np.random.randint(0, 4)
#             #         X_img = np.rot90(X_img, k=n_rotate, axes=(1, 2))
#             if self.aug:
#                 X_img = seq_aug(image=X_img)
#                 # X_img = preprocess_resnet(X_img)
#         return X_img

#     def _load_label(self, obs):
#         batch_adata = self.adata[obs, self.genes].copy()

#         return tuple([batch_adata.to_df()[i].values for i in self.genes])
    
#     def _load_position(self, obs):
#         return self.adata.obs.loc[obs, "X"].values, self.adata.obs.loc[obs, "Y"].values
        
        
#     def get_classes(self):
#         return self.adata.to_df().loc[:, self.genes]




In [96]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.preprocessing import image
from stimage._imgaug import seq_aug
from sklearn.neighbors import KDTree
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, adata, dim=(299, 299), n_channels=3, genes=None, aug=False, tile_path="tile_path"):
        'Initialization'
        self.dim = dim
        self.adata = adata
        self.n_channels = n_channels
        self.genes = genes
        self.num_genes = len(genes)
        self.aug = aug
        self.tile_path = tile_path
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(self.adata.n_obs)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Find list of IDs
        obs_temp = self.adata.obs_names[index]

        # Generate data
        X_img = self._load_img(obs_temp)
        y = self._load_label(obs_temp)
        pos_x,pos_y = self._load_position(obs_temp)
        
        return {"input_img":X_img, "x":pos_x, "y":pos_y}, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.adata.n_obs)

    def _load_img(self, obs):
        img_path = self.adata.obs.loc[obs, 'tile_path']
        X_img = image.load_img(img_path, target_size=self.dim)
        X_img = image.img_to_array(X_img).astype('uint8')
        #         X_img = np.expand_dims(X_img, axis=0)
        #         n_rotate = np.random.randint(0, 4)
        #         X_img = np.rot90(X_img, k=n_rotate, axes=(1, 2))
        if self.aug:
            X_img = seq_aug(image=X_img)
#         X_img = preprocess_resnet(X_img)
        return X_img

    def _load_label(self, obs):
        batch_adata = self.adata[obs, self.genes].copy()

        return tuple([batch_adata.to_df()[i].values for i in self.genes])
    
    def _load_position(self, obs):
        return int(self.adata.obs.loc[obs, "X"]), int(self.adata.obs.loc[obs, "Y"])

    def get_classes(self):
        return self.adata.to_df().loc[:, self.genes]

In [97]:
# i = int(sys.argv[1])
i = 0

test_sample = samples[i]
n_genes = len(gene_list)

adata_all_train_valid = adata_all[adata_all.obs["library_id"].isin(
    adata_all.obs.library_id.cat.remove_categories(test_sample).unique())]

training_index = adata_all_train_valid.obs.sample(frac=0.7, random_state=1).index
training_dataset = adata_all_train_valid[training_index,].copy()

valid_index = adata_all_train_valid.obs.index.isin(training_index)
valid_dataset = adata_all_train_valid[~valid_index,].copy()

test_index = adata_all.obs.library_id == test_sample
test_dataset_1 = adata_all[test_index,].copy()


train_gen = tf.data.Dataset.from_generator(
        lambda:DataGenerator(adata=training_dataset,
                      genes=gene_list, aug=False),
        output_types=({"input_img":tf.float32, "x":tf.float32, "y":tf.float32}, tuple([tf.float32]*n_genes)),
#         output_shapes=([299,299,3],1,1, tuple([1]*n_genes))
)
train_gen_ = train_gen.shuffle(buffer_size=10).batch(32).repeat(1).cache().prefetch(tf.data.experimental.AUTOTUNE)
valid_gen = tf.data.Dataset.from_generator(
        lambda:DataGenerator(adata=valid_dataset,
                      genes=gene_list),
        output_types=({"input_img":tf.float32, "x":tf.float32, "y":tf.float32}, tuple([tf.float32]*n_genes)),
#         output_shapes=(([299,299,3],1,1), tuple([1]*n_genes))
)
valid_gen_ = valid_gen.shuffle(buffer_size=10).batch(32).repeat(1).cache().prefetch(tf.data.experimental.AUTOTUNE)
test_gen_1 = tf.data.Dataset.from_generator(
        lambda:DataGenerator(adata=test_dataset_1,
                      genes=gene_list),
        output_types=({"input_img":tf.float32, "x":tf.float32, "y":tf.float32}, tuple([tf.float32]*n_genes)),
#         output_shapes=(([299,299,3],1,1), tuple([1]*n_genes))
)
test_gen__1 = test_gen_1.batch(1)

In [98]:
# for a, b in train_gen_.take(1):
#     print(a)
#     print(b)

In [99]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [100]:
# class PatchEncoder(layers.Layer):
#     def __init__(self, position_dim, projection_dim):
#         super(PatchEncoder, self).__init__()
#         self.num_patches = position_dim
#         self.projection = layers.Dense(units=projection_dim)
#         self.position_embedding = layers.Embedding(
#             input_dim=position_dim, output_dim=projection_dim
#         )

#     def call(self, patch, x, y):
#         encoded = self.projection(patch) + self.position_embedding(x) + self.position_embedding(y)
# #         encoded = self.position_embedding(x) + self.position_embedding(y)
#         return encoded

In [101]:
# class PatchEncoder(layers.Layer):
#     def __init__(self, position_dim, projection_dim):
#         super(PatchEncoder, self).__init__()
#         self.num_patches = position_dim
#         self.projection = layers.Dense(projection_dim, input_shape=(299,299,3), activation=None)
#         self.position_embedding_x = layers.Embedding(
#             input_dim=position_dim, output_dim=projection_dim
#         )
#         self.position_embedding_y = layers.Embedding(
#             input_dim=position_dim, output_dim=projection_dim
#         )

#     def call(self, patch, x, y):
#         encoded = self.projection(patch) + self.position_embedding_x(x) + self.position_embedding_y(y)
# #         encoded = self.position_embedding(x) + self.position_embedding(y)
#         return encoded

In [102]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [107]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
def ViT_NB(tile_shape, pos_shape, n_genes=None, projection_dim=64, num_patches=24, transformer_layers=8,
           mlp_head_units = [256, 128], num_heads = 4, patch_size=13):
    tile_inputs = layers.Input(shape=tile_shape,name="input_img")
    x = layers.Input(shape=pos_shape, name="x")
    y = layers.Input(shape=pos_shape, name="y")
#     tile_emb = layers.Flatten()(tile_inputs)
#     resnet_base = ResNet50(weights='imagenet', include_top=False, pooling="max")
#     for i in range(len(resnet_base.layers)):
#         resnet_base.layers[i].trainable = False
#     tile_features = resnet_base(tile_inputs)
    
    num_patches = (299 // patch_size) ** 2
    patches = Patches(patch_size)(tile_inputs)
    
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    transformer_units = [
        projection_dim*2,
        projection_dim,
    ]

    for _ in range(transformer_layers):

        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1,x1)

        x2 = layers.Add()([attention_output, encoded_patches])

        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        encoded_patches = layers.Add()([x3, x2])


    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
#     representation = layers.GlobalAveragePooling1D()(representation)
    representation = layers.Flatten()(representation)
#     representation = layers.Dropout(0.5)(representation)

    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.1)

    output_layers = []
    for i in range(n_genes):
        output = layers.Dense(2)(features)
        output_layers.append(layers.Lambda(negative_binomial_layer, name="gene_{}".format(i))(output))

    model = Model(inputs=[tile_inputs, x, y], outputs=output_layers)
    #     losses={}
    #     for i in range(8):
    #         losses["gene_{}".format(i)] = negative_binomial_loss(i)
    #     optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.Adam(1e-5)
    model.compile(loss=negative_binomial_loss,
                  optimizer=optimizer)
    return model

In [108]:
K.clear_session()
model = ViT_NB(tile_shape=(299,299,3), pos_shape=1, n_genes=n_genes)

In [109]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
patches (Patches)               (None, None, 507)    0           input_img[0][0]                  
__________________________________________________________________________________________________
patch_encoder (PatchEncoder)    (None, 529, 64)      66368       patches[0][0]                    
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 529, 64)      128         patch_encoder[0][0]              
______________________________________________________________________________________________

In [110]:
model.fit(train_gen_, epochs=1)

272/272 [==============================] - 403s 1s/step - loss: 21.0997 - gene_0_loss: 1.3475 - gene_1_loss: 0.9480 - gene_2_loss: 1.6502 - gene_3_loss: 1.5688 - gene_4_loss: 1.4606 - gene_5_loss: 1.1574 - gene_6_loss: 1.5859 - gene_7_loss: 1.7486 - gene_8_loss: 1.6521 - gene_9_loss: 1.4427 - gene_10_loss: 1.7358 - gene_11_loss: 1.8393 - gene_12_loss: 1.4009 - gene_13_loss: 1.5619


In [112]:
test_predictions = model.predict(test_gen__1)

In [113]:
from scipy.stats import nbinom
y_preds = []
for i in range(n_genes):
    n = test_predictions[i][:, 0]
    p = test_predictions[i][:, 1]
    y_pred = nbinom.mean(n, p)
    y_preds.append(y_pred)
test_dataset_1.obsm["predicted_gene"] = np.array(y_preds).transpose()

In [115]:
df = pd.DataFrame()

In [116]:
test_dataset_1_ = test_dataset_1[:,gene_list].copy()
test_dataset_1_.X = test_dataset_1_.obsm["predicted_gene"]

pred_adata = test_dataset_1_
test_dataset = test_dataset_1

for gene in pred_adata.var_names:
    cor_val = calculate_correlation(pred_adata.to_df().loc[:,gene], test_dataset.to_df().loc[:,gene])
    df = df.append(pd.Series([gene, cor_val, test_sample, "STimage"], 
                         index=["Gene", "Pearson correlation", "Slide", "Method"]),
                  ignore_index=True)

In [118]:
df["Pearson correlation"].mean()

0.23832510908865237